In [3]:
import requests
from selectolax.parser import HTMLParser
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor
import os
from bs4 import BeautifulSoup as bs

In [ ]:
payload = ""
headers = {"cookie": "XSRF-TOKEN=eyJpdiI6InlrWWtpam1qRUlGazdOV0hGTjNvbGc9PSIsInZhbHVlIjoiQ2J3OHVMaUd2Zno5aW5EQkMxYXhNdGhmQkZ5YXZZNVhTeTFFR3pJL2VLK0lEaloxMWJnTmNSMGVXRmRXSWM2QlNURHdFVEc5N0FsektaQ2FOYzkySVJYTHNFY1JQSzVITUIrYi8rZGhJcWFnZWY4Z21rbUNjcnc4eWM1UnpBUE0iLCJtYWMiOiI0NzFhMDg5YTA2ZmRhNmNiNmUyN2RmODljNDc3MTIxMmJhODAyMTFjOTM0NDhkM2RlOWYzY2YzNmJmZTdhMGEyIiwidGFnIjoiIn0%253D; laravel_session=eyJpdiI6ImIvU0RWbEh6YnNOUDB6NmJMajVoRVE9PSIsInZhbHVlIjoiOWZLR3dOaWhrK1JjRU5nMUMzYWNNa09oWG4wTEN3RitNMjJ3elZTYkJmTHZINGNtQ0hrYTVpVW5HdFlYdHRsTzJ0TlRIV2V5MnJIN296UnJaeE9uRkIvMUpTYnQvWGtCcjNjcGdMKzJhNDVRTEhqMVJyNVhGY0pqQkduMnJEZGgiLCJtYWMiOiJhNzZlOTdkNTljODcyYzhjOTAyYWViMDU5MmRhYWI1MTA2ODIxYzUxMmMxNTI2MmFkZDQ3OGUxNzk2NTczN2U5IiwidGFnIjoiIn0%253D; bc=eyJpdiI6IkJJb1pGTGFuL0VDSGNETm9hb05mRkE9PSIsInZhbHVlIjoiUTV2bmxuWGJYYzhQVi9JMjhWazFvMDllOGMrUHQxdzhSTjNTa3NuS0pMd3dRUDlpREI4aGdZSUFqbTN6U0lLZDRwS3BTeS9ETTNQVUV2blI5V2ZhUVhlWXhYQVk3cExUM0lISUJjbVMyZTg9IiwibWFjIjoiMjlmOTIxODg3MDk2MjY3MjZhYjU0NzA2YTVjOWY5MzQwNzdhZWI2YWVmYzEzNTI1NTIzNTI1OTczNmRhNThjZCIsInRhZyI6IiJ9"}

# response = requests.request("GET", url, data=payload, headers=headers)

## 建立廠牌連結清單
- 返回brand_link_list以便後續可以調用

In [ ]:
url_brand_menu = "https://autos.yahoo.com.tw/new-cars/research"
response = requests.get(url_brand_menu, headers=headers)
parser = HTMLParser(response.text)
brand_link_list = [brand.css_first("a.gabtn").attributes["href"] for brand in parser.css("div.list")]

## 根據連結清單來找出每個年份、車型的連結
- 在編寫function的時候可以加入parameter去控制要爬取的年份會比較節省<font color=red>**資源**</font>

In [ ]:
model_link_list = []

for link in brand_link_list:
    response = requests.get(link, headers=headers)
    parser = HTMLParser(response.text)
    model_links = parser.css("div.body-wrapper > div.main > div.make-main.jq-make-wrapper > div:nth-child(n+3) > a.gabtn")
    for model_link in model_links:
        model_link_list.append(model_link.attributes["href"])  
    

In [ ]:
response = requests.get("https://autos.yahoo.com.tw/new-cars/trim/audi-a3-sportback-2018-35-tfsi/spec", headers=headers)

In [ ]:
parser = HTMLParser(response.text)
values = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n)")
keys = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n-1)")

In [ ]:
target_info = ["動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"]
info_dict = dict()
for key, value in zip(keys, values):
    print(key.text(), value.text())
    if key.text().strip() in target_info:
        info_dict.setdefault(key.text().strip(), value.text().strip())

In [ ]:
result = list(info_dict.values())
if "排氣量" not in info_dict.keys():
    result.insert(1, "NA")
if "車重" not in info_dict.keys():
    result.append("NA")

In [ ]:
brand = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(5)").text()
year = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes["title"][0:4]
series = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes["title"][5:]
model = parser.css_first("a.disabled").text()
price = parser.css_first("div.body-wrapper > div.main > div.trim-main > h3 > span:nth-child(1) > font").text().strip()

In [ ]:
s = [year, brand, series, model, price] + result

In [ ]:
s

In [ ]:
df = pd.DataFrame(columns=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])

In [ ]:
data = pd.Series(s, index=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])

In [ ]:
df = df.append(data, ignore_index=True)

In [ ]:
df

In [ ]:
df["前輪尺寸"] = df["輪胎尺碼"].str.split(" ", expand=True)[0]
df["後輪尺寸"] = df["輪胎尺碼"].str.split(" ", expand=True)[1]

In [ ]:
df[["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "車身型式", "車重", "前輪尺寸", "後輪尺寸"]]

In [ ]:
response = requests.get("https://autos.yahoo.com.tw/new-cars/model/toyota-corolla-altis-2020", headers=headers)

In [ ]:
parser = HTMLParser(response.text)

In [ ]:
for i in parser.css("div.body-wrapper > div.main > div.main-model > div.model-wrapper > ul > a"):
    print(i.attributes["href"])

In [ ]:
url = "https://autos.yahoo.com.tw/new-cars/trim/audi-a4-sedan-2023-40-tfsi%E4%BA%AB%E9%A6%AD%E7%89%88/spec"

In [ ]:
res = requests.get(url, headers=headers)
try:
    for data in res.iter_content(chunk_size=1024):
        data
except:
    print("Failed")
    pass

parser = HTMLParser(res.text)
values = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n)")
keys = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n-1)")
target_info = ["動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"]
info_dict = dict()
for key, value in zip(keys, values):
    # print(key.text(), value.text())
    if key.text().strip() in target_info:
        info_dict.setdefault(key.text().strip(), value.text().strip())
result = list(info_dict.values())
print(result)
if "排氣量" not in info_dict.keys():
    result.insert(1, "NA")
if "車重" not in info_dict.keys():
    result.append("NA")
try:
    brand = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(5)").text()
except:
    brand = "NA"
try:
    year = \
        parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
            "title"][0:4]
except:
    year = "NA"
try:
    series = \
        parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
            "title"][5:]
except:
    series = "NA"
try:
    model = parser.css_first("a.disabled").text()
except:
    model = "NA"
try:
    price = parser.css_first(
        "div.body-wrapper > div.main > div.trim-main > h3 > span:nth-child(1) > font").text().strip()
except:
    price = "NA"
data = [year, brand, series, model, price] + result
print(data)
try:
    row = pd.Series(data,
                    index=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])
except ValueError:
    # with open("fail_extract_log.csv", "a", newline="") as f:
    #     writer = csv.writer(f)
    #     writer.writerow([url, result, data])
    pass
else:
    print(row)
    df = df.append(row, ignore_index=True)

In [ ]:
df.drop_duplicates(inplace=True)
df

In [ ]:
df = pd.DataFrame(columns=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])

In [ ]:
class RetrieveData:
    def __init__(self):
        self.fail_log = self.logCsvToList()
        self.df = pd.DataFrame(columns=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])
    
    def logCsvToList(self):
        fail_list = pd.read_csv(r"C:\Users\kc.hsu\Python workspace\sales_related\測試區\fail_extract_log.csv", header=None)[0].values.tolist()
        return fail_list
    
    def retrieve(self):
        for link in self.fail_log:
            res = requests.get(link, headers=headers)
            try:
                for data in res.iter_content(chunk_size=1024):
                    data
            except:
                print("Failed")
                pass
            else:
                parser = HTMLParser(res.text)
                values = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n)")
                keys = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n-1)")
                target_info = ["動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"]
                info_dict = dict()
                for key, value in zip(keys, values):
                    # print(key.text(), value.text())
                    if key.text().strip() in target_info:
                        info_dict.setdefault(key.text().strip(), value.text().strip())
                result = list(info_dict.values())
                print(result)
                if "排氣量" not in info_dict.keys():
                    result.insert(1, "NA")
                if "車重" not in info_dict.keys():
                    result.append("NA")
                try:
                    brand = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(5)").text()
                except:
                    brand = "NA"
                try:
                    year = \
                        parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
                            "title"][0:4]
                except:
                    year = "NA"
                try:
                    series = \
                        parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
                            "title"][5:]
                except:
                    series = "NA"
                try:
                    model = parser.css_first("a.disabled").text()
                except:
                    model = "NA"
                try:
                    price = parser.css_first(
                        "div.body-wrapper > div.main > div.trim-main > h3 > span:nth-child(1) > font").text().strip()
                except:
                    price = "NA"
                data = [year, brand, series, model, price] + result
                print(data)
                try:
                    row = pd.Series(data,
                                    index=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])
                except ValueError:
                    pass
                else:
                    print(row)
                    self.df = self.df.append(row, ignore_index=True)
        return self.df
    
    def reshape(self):
        self.df["前輪尺寸"] = self.df["輪胎尺碼"].str.split(" ", expand=True)[0]
        self.df["後輪尺寸"] = self.df["輪胎尺碼"].str.split(" ", expand=True)[1]
        self.df = self.df[["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "車身型式", "車重", "前輪尺寸", "後輪尺寸"]]
        return self.df
    
    
        
    
    

In [ ]:
result = RetrieveData()
result = result.retrieve().reshape()
result

## 測試多線程

In [ ]:
file_size = os.stat(r"C:\Users\kc.hsu\Python workspace\sales_related\測試區\fail_extract_log.xlsx").st_size

In [ ]:
if file_size > 0:
    

In [ ]:
links = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\測試區\fail_extract_log.xlsx", header=None)[0].values.tolist()

In [ ]:
df = pd.DataFrame(columns=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])

In [ ]:
def retrieve(url):
    res = requests.get(url, headers=headers)
    try:
        for data in res.iter_content(chunk_size=1024):
            data
    except:
        print("Failed")
        pass
    else:
        parser = HTMLParser(res.text)
        values = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n)")
        keys = parser.css("div.body-wrapper > div.main > div.trim-main > div.trim-spec-detail > div:nth-child(1) > ul > li > span:nth-child(2n-1)")
        target_info = ["動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"]
        info_dict = dict()
        for key, value in zip(keys, values):
            # print(key.text(), value.text())
            if key.text().strip() in target_info:
                info_dict.setdefault(key.text().strip(), value.text().strip())
        result = list(info_dict.values())
        # print(result)
        if "排氣量" not in info_dict.keys():
            result.insert(1, "NA")
        if "車重" not in info_dict.keys():
            result.append("NA")
        try:
            brand = parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(5)").text()
        except:
            brand = "NA"
        try:
            year = \
                parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
                    "title"][0:4]
        except:
            year = "NA"
        try:
            series = \
                parser.css_first("div.body-wrapper > div.main > div.bread > span > a:nth-child(7)").attributes[
                    "title"][5:]
        except:
            series = "NA"
        try:
            model = parser.css_first("a.disabled").text()
        except:
            model = "NA"
        try:
            price = parser.css_first(
                "div.body-wrapper > div.main > div.trim-main > h3 > span:nth-child(1) > font").text().strip()
        except:
            price = "NA"
        data = [year, brand, series, model, price] + result
        # print(data)
        try:
            row = pd.Series(data,
                            index=["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "輪胎尺碼", "車身型式", "車重"])
        except ValueError:
            pass
        else:
            # print(row)
            # df = df.append(row, ignore_index=True)
            return row

In [ ]:
with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(retrieve, links)
for r in results:
    # print(r)
    df = df.append(r, ignore_index=True)
df["前輪尺寸"] = df["輪胎尺碼"].str.split(" ", expand=True)[0]
df["後輪尺寸"] = df["輪胎尺碼"].str.split(" ", expand=True)[1]
df = df[["年份", "品牌", "車系", "車型", "價格", "動力型式", "排氣量", "驅動型式", "車身型式", "車重", "前輪尺寸", "後輪尺寸"]]
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

In [ ]:
yahoo = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\測試區\yahoo_car_oe_spec.xlsx")

In [ ]:
final_yahoo = pd.concat([yahoo, df]).drop_duplicates().reset_index(drop=True)

In [ ]:
final_yahoo.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\Yahoo汽車OE size.xlsx", index=False)

In [ ]:
v = ["1", "2", "3"]
k = ["a", "b", "c"]

In [ ]:
{"test" + str(k): v for k, v in zip(k,v)}

In [ ]:
{k: v for k, v in zip(k, v)}